# Paper Toss Clone

## Background

This code was based off of the IOS game Paper Toss where the objective of the game was to throw a paper ball into a trash can with a fan blowing wind left or right and the speed of the wind was given to the player.

## Equations used:

$$\frac{dx}{dt} = v_x \qquad \frac{dy}{dt} = v_y \qquad \frac{dz}{dt} = v_z$$

$$\frac{d v_x}{dt} = \frac{F_{net,x}}{m} \qquad \frac{d v_y}{dt} = \frac{F_{net,y}}{m} \qquad \frac{d v_y}{dt} = \frac{F_{net,y}}{m}$$

Used to find the motion of the ball where Fnet is the sum of all forces

Intertial Drag: $$\vec{F}_2 = -\frac{1}{2}C_d \rho A v^2 \hat{v}$$

where $C_d$ is the drag coefficient, $\rho$ is the density of the fluid, and $A$ is the cross-sectional area of the object.


Used to find the total Force of a sphere traveling through air: $$\vec{F}_d = -b_1|\vec{v}|\hat{v} + -b_2|\vec{v}|^2\hat{v} + \cdots$$

We can neglect higher order terms.

## My Version

After watching mulitple videos of the game and playing the game myself when I was younger I realized that the player could not throw the ball faster which meant that the velocity for the ball is fixed and that it never changes. However, the only variables that do change in the game are the angle at which the player throws the ball by swiping with their finger and the speed and direction that the fan blows. Because I realized this, all I had to do was create functions to input a random wind in a direction and prompt the user to enter a angle to throw the ball. I also had to use a function from previous code and calculate the variable drag of the ball including gravity and the wind.

## Assumptions: 
- Values used in program such as inital speed, wind speed, travel distance, and the size of the 'trash can' are scaled to work with the simulation.

# Code:

In [1]:
import ode #ode.py should be in the same folder as your notebook
import numpy as np
import matplotlib.pyplot as plt
import random as rand
from vpython import *

<IPython.core.display.Javascript object>

In [2]:

g = 9.8 #N/kg

rho = 1.2 #kg/m^3

mu = 1.8e-5 #kg/m/s

r = 74e-3/2 #74 mm diameter, 9.25" in circumference

A = np.pi*r**2 #cross-sectional area

Cd = 0.47 #actually depends on speed

m = 0.1 #kg

b2 = 1/2*Cd*rho*A

In [3]:
def quaddrag(d, t):
    global v,vxwind,vywind,vzwind, vxquad,vyquad,vzquad
    """ Calculate and return the derivative of each quantity in an array d at the time t.
    
    Keyword arguments:
    t -- time at the beginning of the time step
    d -- an array of variables at time t
    """
    
    x = d[0]
    y = d[1]
    z = d[2]
    vx = d[3]
    vy = d[4]
    vz = d[5]
    
    dxdt = vx 
    dydt = vy
    dzdt = vz

    
    vxquad = vx - vxwind
    vyquad = vy - vywind
    vzquad = vz - vzwind
    
    
    v = np.sqrt(vxquad**2 + vyquad**2 + vzquad**2)
    
    #drag components:
    a = 0.36
    b = 0.14
    c = 0.27
    vc = 34
    chi = (v-vc)/4
    
    
    if chi < 0:
        Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2)
    else:
        Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2/4)
        
    b2 = 1/2*Cd*rho*A
    
    Fdragx = -b2*v**2*vxquad/v
    Fdragy = -b2*v**2*vyquad/v
    Fdragz = -b2*v**2*vzquad/v
    
    Fgravy = -m*g
    
    Fnetx = Fdragx
    Fnety = Fgravy + Fdragy 
    Fnetz = Fdragz
    
    dvxdt = Fnetx/m
    dvydt = Fnety/m
    dvzdt = Fnetz/m
    
    derivs = np.array([dxdt,dydt,dzdt, dvxdt, dvydt,dvzdt])
    
    return derivs

The function titled "quaddrag" calculates the total drag using all of the forces possible for the motion of a sphere. Without the calculations of drag being added into the simulation the ball would fly much further than expected and therefore would not display the desired results that you would normally want in a realistic video game. 

In [8]:
#CONSTANTS
def math(phideg):
    global b2,m,g,r


    g = 9.8 #N/kg
    
    rho = 1.2 #kg/m^3
    
    mu = 1.8e-5 #kg/m/s
    
    r = 74e-3/2 #74 mm diameter, 9.25" in circumference
    
    A = np.pi*r**2 #cross-sectional area
    
    Cd = 0.47 #actually depends on speed
    
    m = 0.1 #kg
    
    b2 = 1/2*Cd*rho*A


    vmag0mph = 25 #inital speed, does not change
    thetadeg = 60 #pre defined angle(deg)

    vmag0 = vmag0mph*0.44704 #convert mph to m/s
    theta = thetadeg*np.pi/180 #convert deg to rad
     #Inital angle paper is thrown(USER INPUT)

    phi = phideg*np.pi/180#deg --> radians

    #inital components 
    x = 0
    y = 0
    z = 0

    vy = vmag0*np.sin(theta)
    vx = vmag0*np.cos(theta)*np.sin(phi)
    vz = -vmag0*np.cos(theta)*np.cos(phi)


    t = 0
    h = 1e-4


    # create array:
    data = np.array([x, y , z, vx, vy, vz])

    #create list:
    xlist=[]
    ylist=[]
    zlist=[]
    tlist=[]

    #append list:
    xlist.append(x)
    ylist.append(y)
    zlist.append(z)
    tlist.append(t)

    while y >= 0:

        data = ode.RK4(quaddrag, data, t, h)
        t = t + h

        x = data[0]
        y = data[1]
        z = data[2]

        tlist.append(t)
        xlist.append(x)
        ylist.append(y)
        zlist.append(z)
    #print("total drag", np.sqrt(xlist[-1]**2+ylist[-1]**2+zlist[-1]**2))
    return xlist, ylist, zlist

The function titled "math" calulcates the toal drag and returns the lists used to properlly simulate the flight of the ball while taking into account the wind and gravity.

In [9]:
def wind():
    num = int(np.random.uniform(1,140))
    if num < 25 and num>0:
        wind = 10
    elif num>25 and num<50:
        wind = -10
    elif num > 50 and num < 100:
        wind = 5
    elif num > 80 and num < 100:
        wind = -5
    elif num >100 and num < 120:
        wind = 7
    elif num > 120 and num < 140:
        wind = -7
    return wind

The function titled "wind" is used to pick a random number from 1-140 and uses "if", "else" to return a velocity for wind depending on what number is chosen from 1-140

In [10]:
def animate():
    global vxwind,vywind,vzwind
    scene = canvas(title = "Paper Toss")
    
    winddir = wind()
    
    vxwind = winddir
    vywind = 0
    vzwind = 0
    windvec = vec(vxwind,vywind,vzwind)
    R = 1 #radius of ring
    r = R/3 #radius of ball

    m = 0.1
    
    circ = ring(pos=vec(0,0,-9), axis=vec(0,6,0), radius=R, thickness=R/10, color=color.orange, opacity=0.5)

    ball = sphere(pos = vec(0,0,0), radius = r, color=color.magenta, make_trail= True)
    
    point = False

    while(point == False):
        scene.pause()
        print("wind direction(+,-), speed(m/s):", winddir)
        angle = int(input("Enter your angle:"))
        bx, by, bz = math(angle)
        size = np.size(by)


        for n in range(1,size):
            rate(6000)
            ball.pos=vec(bx[n],by[n],bz[n])
            ds = np.sqrt((bx[n]-circ.pos.x)**2+(bz[n]-circ.pos.z)**2)
            if ds < R:
                point = True
        if point == True:
            print("score")
        else:
            print("try again")

The function titled "animate" is the main function of the entire code and uses all of the functions above to simulate the game itself. I also integrated a while loop to keep the game going until the player can pick the correct angle to score a point. Once the player succesfully scores a point the game ends and they must re-run the code to play once again.

# Directions: 
once prompted with a wind direction($'+'$ meaning to the right and $'-'$ meaning to the left) input a the angle at which you would like the ball to be thrown, again $'-'$ meaning to the left and $'+'$ meaning to the right.

In [11]:
animate()

<IPython.core.display.Javascript object>

wind direction(+,-), speed(m/s): 5
Enter your angle:-10
score


## Conclusion:

Overall, my version of paper toss is a much more realistic version because it takes into account the drag of the sphere and would preform more like how it would work in real life. The version on the IOS does not take drag forces into account.